# Spurious Correlations in Highly Dimensional Big Data

This Notebook aims at showing how PCA and Random Projection can solve the problem of spurious correlations in Big Data.

In [15]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.random_projection import johnson_lindenstrauss_min_dim
from sklearn import random_projection

## DataFrame Creation

In [2]:
# Number of columns
w = 100000

# Number of rows for df
x = 10000 

# Number of rows for df2
y = 1000

# Number of rows for df2
z = 100

# We fix the seed so that random number do not change everytime the script is launched
np.random.seed(0)

# Creating a dataframe with x number of rows and y number of columns
df = pd.DataFrame(np.random.random_sample((x,w)))

# Shuffle rows of dataframe
df = df.sample(frac=1)
df

,0,1,2,3,4,5,6,7,8,9,...,99990,99991,99992,99993,99994,99995,99996,99997,99998,99999
5777,0.013107,0.470582,0.782615,0.356561,0.713228,0.001639,0.531202,0.149542,0.238533,0.156730,...,0.014578,0.703602,0.121921,0.809777,0.141529,0.051837,0.615442,0.598923,0.463951,0.278275
3783,0.960434,0.443701,0.571535,0.813105,0.448731,0.292695,0.378250,0.988330,0.737386,0.240126,...,0.865072,0.048426,0.556778,0.939422,0.430007,0.304750,0.629446,0.784474,0.027791,0.411822
1295,0.145563,0.462142,0.170783,0.475111,0.200657,0.647372,0.260204,0.386845,0.420123,0.275254,...,0.038677,0.744098,0.731141,0.329909,0.280043,0.441809,0.997663,0.531609,0.461333,0.930565
838,0.415516,0.027677,0.297519,0.258756,0.415945,0.530397,0.204964,0.330645,0.675209,0.114403,...,0.305023,0.463645,0.621614,0.614746,0.401539,0.840944,0.074929,0.523992,0.074633,0.005774
1786,0.838305,0.187315,0.668331,0.625653,0.694821,0.305795,0.505483,0.220637,0.973320,0.472144,...,0.863375,0.903354,0.090747,0.823878,0.833744,0.254442,0.854606,0.282577,0.124343,0.716000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3408,0.998848,0.085800,0.471002,0.409039,0.700793,0.540194,0.107416,0.102830,0.798725,0.773601,...,0.026062,0.044855,0.712999,0.918720,0.893730,0.286155,0.116346,0.535174,0.483603,0.914082
6336,0.662911,0.213004,0.723441,0.183560,0.722360,0.145670,0.330431,0.135678,0.996597,0.220519,...,0.144963,0.446426,0.148142,0.398983,0.215387,0.081686,0.622973,0.427959,0.442991,0.751608
1880,0.416949,0.271127,0.791953,0.508140,0.039867,0.257467,0.701436,0.249736,0.430972,0.918784,...,0.435077,0.629779,0.088950,0.311274,0.453839,0.558268,0.585202,0.938641,0.548909,0.466086
763,0.647500,0.127408,0.799678,0.626792,0.124441,0.137988,0.041683,0.283041,0.592727,0.516547,...,0.854014,0.113504,0.449887,0.768145,0.033564,0.712753,0.263060,0.514465,0.901452,0.073480


In [3]:
# Pass table header to latex
df.head().to_latex(index=False,columns=[1,2,99998,99999], caption="Dataframe", label="Datraframe")

'\\begin{table}\n\\centering\n\\caption{Dataframe}\n\\label{Datraframe}\n\\begin{tabular}{rrrr}\n\\toprule\n    1     &     2     &     99998 &     99999 \\\\\n\\midrule\n 0.470582 &  0.782615 &  0.463951 &  0.278275 \\\\\n 0.443701 &  0.571535 &  0.027791 &  0.411822 \\\\\n 0.462142 &  0.170783 &  0.461333 &  0.930565 \\\\\n 0.027677 &  0.297519 &  0.074633 &  0.005774 \\\\\n 0.187315 &  0.668331 &  0.124343 &  0.716000 \\\\\n\\bottomrule\n\\end{tabular}\n\\end{table}\n'

In order to assess the correlations of the different parameters, correlations between the column with index 0 and the remaining 99998 other columns is assessed.

In [17]:
# Assigning X to all columns except 0
X_df = df.drop(columns=0)
X_df.head()

# Assigning Y to column 0
Y_df = df[0]

# The following line makes Y become a list
Y_df = np.array(Y_df).reshape(-1)
print(X_df.shape,Y_df.shape)

# Calculate correlations between selected column and column 0
list_titles = X_df.columns
list_corr_df = []
for i in list_titles:
    list_corr_df.append(abs(np.corrcoef(Y_df, X_df[i])[0][1]))

(10000, 99999) (10000,)


To see how correlations between the different parameters changes due to the data size, the original dataframe is compared to sub-set dataframes which only takes the first y or z rows of the original dataframe.

In [5]:
# Creating smaller dataframe taking y number of rows from original dataframe
df2 = df.iloc[:y]

# Assigning X to all columns except 0
X_df2 = df2.drop(columns=0)
X_df2.head()

# Assigning Y to column 0
Y_df2 = df2[0]

# The following line makes Y become a list
Y_df2 = np.array(Y_df2).reshape(-1)
print(X_df2.shape,Y_df2.shape)

list_titles = X_df2.columns

# Calculate correlations between selected column and column 0
list_corr_df2 = []
for i in list_titles:
    list_corr_df2.append(abs(np.corrcoef(Y_df2, X_df2[i])[0][1]))

(1000, 99999) (1000,)


In [6]:
# Creating smaller dataframe taking z number of rows from original dataframe
df3 = df.iloc[:z]

# Assigning X to all columns except 0
X_df3 = df3.drop(columns=0)
X_df3.head()

# Assigning Y to column 0
Y_df3 = df3[0]

# The following line makes Y become a list
Y_df3 = np.array(Y_df3).reshape(-1)
print(X_df3.shape,Y_df3.shape)

list_titles = X_df3.columns

# Calculate correlations between selected column and column 0
list_corr_df3 = []
for i in list_titles:
    list_corr_df3.append(abs(np.corrcoef(Y_df3, X_df3[i])[0][1]))

(100, 99999) (100,)


We now compare which of the dataframes has the highest proportions of correlations.

In [8]:
a = 0
b = 0
c = 0

for i in range(0,len(list_corr_df)):
    if abs(list_corr_df[i]) > 0.01:
        a+=1
    else:
        pass
        
for i in range(0,len(list_corr_df2)):
    if abs(list_corr_df2[i]) > 0.01:
        b+=1
    else:
        pass
    
for i in range(0,len(list_corr_df3)):
    if abs(list_corr_df3[i]) > 0.01:
        c+=1
    else:
        pass
    
print("Check number of columns df:", len(X_df.columns))
print("Check number of rows df:", len(X_df))
print("Percentage of correlations in df:",(a/len(list_corr_df)*100),"%")
print()
print("Check number of columns df2:", len(X_df2.columns))
print("Check number of rows df2:", len(X_df2))
print("Percentage of correlations in df2:",(b/len(list_corr_df2)*100),"%")
print()
print("Check number of columns df3:", len(X_df3.columns))
print("Check number of rows df3:", len(X_df3))
print("Percentage of correlations in df3:",(c/len(list_corr_df3)*100),"%")

Check number of columns df: 99999
Check number of rows df: 10000
Percentage of correlations in df: 31.997319973199733 %

Check number of columns df2: 99999
Check number of rows df2: 1000
Percentage of correlations in df2: 75.16475164751647 %

Check number of columns df3: 99999
Check number of rows df3: 100
Percentage of correlations in df3: 92.07692076920769 %


Clearly it is seen that the lower the number of observations, the higher the frequency of spurious correlations. However the number of spurious correlations present in the bigger DataFrame is still very significant. This would mean that the dimensionality of the data also plays an important role for the precense of spurious correlations.

To counter spurious correlations, Principal Component Analysis and Random Projection can be used.

## Principal Component Analysis

In PCA, it is up to the user to choose how many dimension the final data has. This is because the explained variance tells you how much information (variance) can be attributed to each of the principal components. This is important as while you can convert a y dimensional space to 2 dimensional space, you lose some of the variance (information) when you do this. It is up to the user to decide how much variance he is ready to lose. 

In [16]:
# Here we pass the daframe of y columns in 2 dimensions
pca = PCA(n_components = 2)
df_pca = pd.DataFrame(pca.fit_transform(df))

# How much of the information is kept by the new dataframe?
print(pca.explained_variance_ratio_)

[0.00015985 0.00015956]


In [17]:
print("Sum of ratios for 2 n_components:",pca.explained_variance_ratio_.sum())

Sum of ratios for 2 n_components: 0.00031941487308860604


The new dataframe is not useful as it almost does not capture any variance (only 0.03%). Thus we need more dimensions.

In [18]:
# Here we pass the daframe of y columns in 100 dimensions
pca = PCA(n_components = 100)
df_pca = pd.DataFrame(pca.fit_transform(df))
print(pca.explained_variance_ratio_)

[0.00016387 0.00016353 0.00016323 0.00016313 0.00016284 0.00016265
 0.00016251 0.00016239 0.00016217 0.00016212 0.00016195 0.00016172
 0.0001616  0.00016143 0.00016135 0.00016132 0.00016116 0.00016094
 0.0001608  0.0001607  0.00016066 0.00016045 0.00016043 0.00016028
 0.00016019 0.00016013 0.00015997 0.00015979 0.00015973 0.00015955
 0.00015946 0.00015942 0.00015938 0.00015929 0.00015916 0.00015907
 0.00015902 0.00015891 0.00015885 0.00015867 0.00015856 0.00015849
 0.00015846 0.00015837 0.0001583  0.0001581  0.000158   0.00015794
 0.00015785 0.00015776 0.00015765 0.00015754 0.00015749 0.0001574
 0.00015729 0.00015721 0.0001571  0.00015699 0.00015693 0.00015682
 0.00015672 0.00015661 0.00015654 0.00015652 0.0001564  0.00015627
 0.00015622 0.00015614 0.00015594 0.00015585 0.0001558  0.00015568
 0.00015555 0.00015549 0.00015546 0.00015528 0.00015521 0.00015516
 0.00015499 0.00015491 0.00015481 0.00015472 0.00015463 0.00015448
 0.00015439 0.00015431 0.0001542  0.00015402 0.00015397 0.00015

In [19]:
print("Sum of ratios for 100 n_components:",pca.explained_variance_ratio_.sum())

Sum of ratios for 100 n_components: 0.015780478235050775


The new dataframe explains only 1.63% of the variance. However it seems that everytime we multiple by 10 the number of columns, the variance is multiplied by by 10 too.

In [20]:
# Here we pass the daframe of y columns in 1000 dimensions
pca = PCA(n_components = 1000)
df_pca = pd.DataFrame(pca.fit_transform(df))
print(pca.explained_variance_ratio_)

[0.00016629 0.0001661  0.00016581 0.0001658  0.00016558 0.00016544
 0.00016529 0.00016525 0.00016514 0.00016512 0.00016487 0.00016484
 0.00016475 0.00016462 0.00016456 0.00016449 0.0001644  0.00016422
 0.00016414 0.00016412 0.00016406 0.00016396 0.00016389 0.00016374
 0.00016366 0.00016362 0.00016357 0.00016351 0.00016342 0.00016338
 0.00016332 0.00016317 0.00016312 0.00016303 0.00016299 0.00016293
 0.00016284 0.00016273 0.00016264 0.0001626  0.00016253 0.00016247
 0.00016242 0.00016231 0.00016226 0.00016222 0.00016218 0.00016209
 0.00016201 0.00016196 0.00016188 0.00016185 0.00016182 0.00016174
 0.00016167 0.0001616  0.00016158 0.00016148 0.00016146 0.00016139
 0.00016132 0.00016124 0.00016122 0.00016114 0.00016109 0.00016101
 0.00016096 0.00016095 0.00016092 0.0001608  0.00016077 0.00016076
 0.00016067 0.00016059 0.00016055 0.00016051 0.00016046 0.00016043
 0.00016038 0.00016035 0.00016027 0.00016021 0.00016012 0.00016008
 0.00016005 0.00015995 0.00015994 0.00015982 0.00015979 0.0001

In [21]:
print("Sum of ratios for 1000 n_components:",pca.explained_variance_ratio_.sum())

Sum of ratios for 1000 n_components: 0.14460176836621738


With 1000 Columns, the sum of all the variances is close to 10%, therefore appplying PCA on data with dimension 100000 and changing it to 10000 dimensions should effective as we see that the variance is multipled by 10 everytime the number of n_components is multiplied by 10 which would result with a variance close to 100%.

In [3]:
# Here we pass the daframe of y columns in 1000 dimensions
pca = PCA(n_components = 10000)
df_pca = pd.DataFrame(pca.fit_transform(df))
print(pca.explained_variance_ratio_)

[1.73222709e-04 1.73103907e-04 1.72949852e-04 ... 4.68212754e-05
 4.67843489e-05 2.20717127e-29]


In [10]:
print("Sum of ratios for 10000 n_components:",pca.explained_variance_ratio_.sum())

Sum of ratios for 10000 n_components: 0.9999999999999999


With 10000 Columns, the sum of all the variances is almost equal to 100% explained variance.

In [11]:
df_new = df_pca
# Assigning X to all columns except 0
X_df_new = df_new.drop(columns=0)

# Assigning Y to column 0
Y_df_new = df_new[0]

# The following line makes Y become a list
Y_df_new = np.array(Y_df_new).reshape(-1)
print(X_df_new.shape,Y_df_new.shape)

list_titles = X_df_new.columns

# Calculate correlations between selected column and column 0
list_corr_df_new = []
for i in list_titles[0:len(list_titles)]:
    list_corr_df_new.append(abs(np.corrcoef(Y_df_new, X_df_new[i])[0][1]))

(10000, 9999) (10000,)


In [15]:
a = 0
b = 0
for i in range(0,len(list_corr_df)):
    if abs(list_corr_df[i]) > 0.01:
        a+=1
    else:
        pass
        
for i in range(0,len(list_corr_df_new)):
    if abs(list_corr_df_new[i]) > 0.01:
        b+=1
    else:
        pass

print("Percentage of correlations in df1:",(a/len(list_corr_df)*100),"%")

print("Percentage of correlations in df_new:",(b/len(list_corr_df_new)*100),"%")

Percentage of correlations in df1: 31.997319973199733 %
Percentage of correlations in df_new: 0.0 %


The correlation of the original DataFrame is close to 0 when transformed through PCA.

PCA seems to be able to counter spurious correlations by reducing from dimension 100000 to 10000. However it is extremely computationally expensive therefore a faster method could be used, such a Random Projection.

## Random Projection

In Random Projection, the parameter eps defines how much the Random Projection can deviate from original DataFrame.
The lower the percentage of the parameter the higher the fidelity of the transformed data.
However the higher the fidelity the less the reduction of dimensions.

For example, a reduced data with a distortion of 10% for a dataset of 10000 rows will have 7894 columns to present the same characteristics as the orginal dataframe as shown below

In [16]:
johnson_lindenstrauss_min_dim(n_samples=10000, eps=0.1)

7894

In [3]:
# Pass df in the random projection to create a new reduced DataFrame with eps = 0.1
transformer = random_projection.GaussianRandomProjection(eps = 0.1)
df_new = pd.DataFrame(transformer.fit_transform(df))

# Assigning X to all columns except 0
X_df_new = df_new.drop(columns=0)

# Assigning Y to column 0
Y_df_new = df_new[0]


# The following line makes Y become a list
Y_df_new = np.array(Y_df_new).reshape(-1)
print(X_df_new.shape,Y_df_new.shape)

list_titles = X_df_new.columns

list_corr_df_zero1 = []
for i in list_titles[0:len(list_titles)]:
    list_corr_df_zero1.append(abs(np.corrcoef(Y_df_new, X_df_new[i])[0][1]))

(10000, 7893) (10000,)


In [4]:
# Pass df in the random projection to create a new reduced DataFrame with eps = 0.2
transformer = random_projection.GaussianRandomProjection(eps = 0.2)
df_new = pd.DataFrame(transformer.fit_transform(df))

# Assigning X to all columns except 0
X_df_new = df_new.drop(columns=0)

# Assigning Y to column 0
Y_df_new = df_new[0]

# The following line makes Y become a list
Y_df_new = np.array(Y_df_new).reshape(-1)
print(X_df_new.shape,Y_df_new.shape)

list_titles = X_df_new.columns

list_corr_df_zero2 = []
for i in list_titles[0:len(list_titles)]:
    list_corr_df_zero2.append(abs(np.corrcoef(Y_df_new, X_df_new[i])[0][1]))

(10000, 2124) (10000,)


In [5]:
# Pass df in the random projection to create a new reduced DataFrame with eps = 0.5
transformer = random_projection.GaussianRandomProjection(eps = 0.5)
df_new = pd.DataFrame(transformer.fit_transform(df))

# Assigning X to all columns except 0
X_df_new = df_new.drop(columns=0)

# Assigning Y to column 0
Y_df_new = df_new[0]


# The following line makes Y become a list
Y_df_new = np.array(Y_df_new).reshape(-1)
print(X_df_new.shape,Y_df_new.shape)

list_titles = X_df_new.columns

list_corr_df_zero5 = []
for i in list_titles[0:len(list_titles)]:
    list_corr_df_zero5.append(abs(np.corrcoef(Y_df_new, X_df_new[i])[0][1]))

(10000, 441) (10000,)


In [10]:
# Pass df in the random projection to create a new reduced DataFrame with eps = 0.8
transformer = random_projection.GaussianRandomProjection(eps = 0.8)
df_new = pd.DataFrame(transformer.fit_transform(df))

# Assigning X to all columns except 0
X_df_new = df_new.drop(columns=0)

# Assigning Y to column 0
Y_df_new = df_new[0]


# The following line makes Y become a list
Y_df_new = np.array(Y_df_new).reshape(-1)
print(X_df_new.shape,Y_df_new.shape)

list_titles = X_df_new.columns

list_corr_df_zero8 = []
for i in list_titles[0:len(list_titles)]:
    list_corr_df_zero8.append(abs(np.corrcoef(Y_df_new, X_df_new[i])[0][1]))

(10000, 245) (10000,)


In [20]:
a = 0
b = 0
c = 0
d = 0
e = 0

for i in range(0,len(list_corr_df)):
    if abs(list_corr_df[i]) > 0.01:
        a+=1
    else:
        pass
        
for i in range(0,len(list_corr_df_zero1)):
    if abs(list_corr_df_zero1[i]) > 0.01:
        b+=1
    else:
        pass

for i in range(0,len(list_corr_df_zero2)):
    if abs(list_corr_df_zero2[i]) > 0.01:
        c+=1
    else:
        pass

for i in range(0,len(list_corr_df_zero5)):
    if abs(list_corr_df_zero5[i]) > 0.01:
        d+=1
    else:
        pass

for i in range(0,len(list_corr_df_zero8)):
    if abs(list_corr_df_zero8[i]) > 0.01:
        e+=1
    else:
        pass

print("Percentage of correlations in df:",(a/len(list_corr_df)*100),"%")
print("Percentage of correlations in df_zero1:",(b/len(list_corr_df_zero1)*100),"%")
print("Percentage of correlations in df_zero2:",(c/len(list_corr_df_zero2)*100),"%")
print("Percentage of correlations in df_zero5:",(d/len(list_corr_df_zero5)*100),"%")
print("Percentage of correlations in df_zero8:",(e/len(list_corr_df_zero8)*100),"%")

Percentage of correlations in df: 31.997319973199733 %
Percentage of correlations in df_zero1: 34.23286456353731 %
Percentage of correlations in df_zero2: 34.32203389830508 %
Percentage of correlations in df_zero5: 29.705215419501137 %
Percentage of correlations in df_zero8: 33.46938775510204 %


Random Projection seems to actually increase the number of spurious correlations. Changing the precision of the embeddings through the "eps" parameter does not seem to influence results in any ral positive way. Random Projection seems to be limited as it is found to be enable to lead to valid results for highly dimensional random Big Data. Random Projection is nonetheless useful in reducing the amount of columns in the original dataframe.

## Conclusion

Spurious correlations can be avoided in Big Data thanks to Dimension Reduction methods.

PCA is very precise but extremely computationally expensive due to high dimensionality.

Random Projection is very quick but lacks precision due to trade off between distortion and correlation.

Future research could find another algorithm which is suitable for high dimensional data which is quick, reliable and does not distort the data.